<a href="https://colab.research.google.com/github/Phuwan-Loythala/wthr-de/blob/main/Data_Project_2_Weather_Data_Engineering_by_Phuloydata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

# Define the API URL
api_url = "https://api.weatherapi.com/v1/current.json?key=4bce76cddea847a4b7435305230111"

#API KEYS
params = {
    'q': 'Pathum Thani'
}

# Make a request to the API
response = requests.get(api_url, params=params)

# Check for a successful response (status code 200)
if response.status_code == 200:
    weather_data = response.json()
    print(weather_data)
else:
    print(f"Failed to fetch weather data. Status code: {response.status_code}")

{'location': {'name': 'Pathum Thani', 'region': 'Pathum Thani', 'country': 'Thailand', 'lat': 14.02, 'lon': 100.53, 'tz_id': 'Asia/Bangkok', 'localtime_epoch': 1698835975, 'localtime': '2023-11-01 17:52'}, 'current': {'last_updated_epoch': 1698835500, 'last_updated': '2023-11-01 17:45', 'temp_c': 31.0, 'temp_f': 87.8, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 20, 'wind_dir': 'NNE', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.85, 'precip_in': 0.03, 'humidity': 63, 'cloud': 25, 'feelslike_c': 38.8, 'feelslike_f': 101.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph': 8.6, 'gust_kph': 13.8}}


In [ ]:
import pandas as pd

# Process the weather data (assuming it's a list of dictionaries)

# Check for a successful response (status code 200)
if response.status_code == 200:
    weather_data = response.json()
    processed_data = []
    # Access the relevant weather information from the JSON response
    country = weather_data['location']['country']
    location_name = weather_data['location']['name']
    temperature = weather_data['current']['temp_c']
    humidity = weather_data['current']['humidity']
    date = weather_data['location']['localtime']

    # Create a dictionary with the extracted data
    processed_data = [{'date': date, 'country': country,'location_name': location_name, 'temperature': temperature, 'humidity': humidity}]

    # Convert the processed data into a DataFrame (using Pandas)
    df = pd.DataFrame(processed_data)

    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    for target_date in df['date']:
      df.append(processed_data, ignore_index=True)
      print(df)
else:
    print(f"Failed to fetch weather data. Status code: {response.status_code}")

                 date   country location_name  temperature  humidity
0 2023-11-01 17:52:00  Thailand  Pathum Thani         31.0        63


<ipython-input-130-7edf8d3fcee7>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(processed_data, ignore_index=True)


In [ ]:
import sqlite3

# Connect to the SQLite database (create one if it doesn't exist)
conn = sqlite3.connect('weather_data.db')

# Create a cursor object
cursor = conn.cursor()

# Create a table to store the data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS weather_data (
        date TEXT,
        temperature REAL,
        humidity REAL
    )
''')

# Insert the data into the table
for _, row in df.iterrows():
    cursor.execute('''
        INSERT INTO weather_data (date, temperature, humidity)
        VALUES (?, ?, ?)
    ''', (row['date'], row['temperature'], row['humidity']))

# Commit the changes and close the database connection
conn.commit()
conn.close()